# TLT YOLO v4 example usecase

This notebook shows an example usecase of YOLO v4 object detection using Transfer Learning Toolkit.

0. [Set up env variables](#head-0)
1. [Prepare dataset and pre-trained model](#head-1) <br>
    1.1 [Download pre-trained model](#head-1-1) <br>
2. [Provide training specification](#head-2)
3. [Run TLT training](#head-3)
4. [Evaluate trained models](#head-4)
5. [Prune trained models](#head-5)
6. [Retrain pruned models](#head-6)
7. [Evaluate retrained model](#head-7)
8. [Visualize inferences](#head-8)
9. [Deploy](#head-9)
10. [Verify deployed model](#head-10)

## 0. Set up env variables <a class="anchor" id="head-0"></a>


In [13]:
# Setting up env variables for cleaner command line commands.
print("Please replace the variable with your key.")
%env GPU_INDEX=0
%env KEY=tlt
%set_env USER_EXPERIMENT_DIR=/workspace/tlt-experiments/insect-thesis
%set_env PRETRAINED_DIR=/workspace/tlt-experiments/insect-thesis/data/faster_rcnn
%set_env DATA_DOWNLOAD_DIR=/workspace/tlt-experiments/insect-thesis/data/kitti/final-test
%env SPECS_DIR=./specs
!mkdir -p $DATA_DOWNLOAD_DIR/yolo_v4

Please replace the variable with your key.
env: GPU_INDEX=0
env: KEY=tlt
env: USER_EXPERIMENT_DIR=/workspace/tlt-experiments/insect-thesis
env: PRETRAINED_DIR=/workspace/tlt-experiments/insect-thesis/data/faster_rcnn
env: DATA_DOWNLOAD_DIR=/workspace/tlt-experiments/insect-thesis/data/kitti/final-test
env: SPECS_DIR=./specs


## 1. Prepare dataset and pre-trained model <a class="anchor" id="head-1"></a>

 We will be using the KITTI detection dataset for the tutorial. To find more details please visit
 http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d. Please download the KITTI detection images (http://www.cvlibs.net/download.php?file=data_object_image_2.zip) and labels (http://www.cvlibs.net/download.php?file=data_object_label_2.zip) to $DATA_DOWNLOAD_DIR.

In [ ]:
# Check the dataset is present
!mkdir -p $DATA_DOWNLOAD_DIR
!if [ ! -f $DATA_DOWNLOAD_DIR/data_object_image_2.zip ]; then echo 'Image zip file not found, please download.'; else echo 'Found Image zip file.';fi
!if [ ! -f $DATA_DOWNLOAD_DIR/data_object_label_2.zip ]; then echo 'Label zip file not found, please download.'; else echo 'Found Labels zip file.';fi

In [ ]:
# unpack 
!unzip -u $DATA_DOWNLOAD_DIR/data_object_image_2.zip -d $DATA_DOWNLOAD_DIR
!unzip -u $DATA_DOWNLOAD_DIR/data_object_label_2.zip -d $DATA_DOWNLOAD_DIR

In [ ]:
# verify
!ls -l $DATA_DOWNLOAD_DIR/

In [4]:
# Generate val dataset out of training dataset
!python3.6 generate_val_dataset.py --input_image_dir=$DATA_DOWNLOAD_DIR/kitti/final-test/train/images \
                                   --input_label_dir=$DATA_DOWNLOAD_DIR/kitti/final-test/train/labels \
                                   --output_dir=$DATA_DOWNLOAD_DIR/kitti/final-test/val

Total 11611 samples in KITTI training dataset
10450 for train and 1161 for val


Additionally, if you have your own dataset already in a volume (or folder), you can mount the volume on `DATA_DOWNLOAD_DIR` (or create a soft link). Below shows an example:
```bash
# if your dataset is in /dev/sdc1
mount /dev/sdc1 $DATA_DOWNLOAD_DIR

# if your dataset is in folder /var/dataset
ln -sf /var/dataset $DATA_DOWNLOAD_DIR
```

In [2]:
# If you use your own dataset, you will need to run the code below to generate the best anchor shape

!yolo_v4 kmeans -l $DATA_DOWNLOAD_DIR/kitti/final-test/train/labels \
                 -i $DATA_DOWNLOAD_DIR/kitti/final-test/train/images \
                 -n 9 \
                 -x 512 \
                 -y 512

# The anchor shape generated by this script is sorted. Write the first 3 into small_anchor_shape in the config
# file. Write middle 3 into mid_anchor_shape. Write last 3 into big_anchor_shape.

Using TensorFlow backend.
Start optimization iteration: 1
Start optimization iteration: 11
Start optimization iteration: 21
Start optimization iteration: 31
Start optimization iteration: 41
Start optimization iteration: 51
Start optimization iteration: 61
Start optimization iteration: 71
Start optimization iteration: 81
Start optimization iteration: 91
Start optimization iteration: 101
Start optimization iteration: 111
Start optimization iteration: 121
Start optimization iteration: 131
Start optimization iteration: 141
Start optimization iteration: 151
Start optimization iteration: 161
Please use following anchor sizes in YOLO config:
(56.00, 66.00)
(131.00, 147.00)
(223.00, 203.00)
(170.91, 293.00)
(327.96, 250.89)
(239.27, 367.87)
(316.00, 406.06)
(406.47, 324.00)
(431.00, 433.10)


### 1.1 Download pre-trained model <a class="anchor" id="head-1-1"></a>

We will use NGC CLI to get the pre-trained models. For more details, go to [ngc.nvidia.com](ngc.nvidia.com) and click the SETUP on the navigation bar.

In [ ]:
!ngc registry model list nvidia/tlt_pretrained_object_detection:*

In [ ]:
!mkdir -p $USER_EXPERIMENT_DIR/pretrained_resnet18/

In [ ]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tlt_pretrained_object_detection:resnet18 --dest $USER_EXPERIMENT_DIR/pretrained_resnet18

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $USER_EXPERIMENT_DIR/pretrained_resnet18/tlt_pretrained_object_detection_vresnet18

## 2. Provide training specification <a class="anchor" id="head-2"></a>
* Augmentation parameters for on-the-fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.
* Whether to use quantization aware training (QAT)

In [6]:
# Provide pretrained model path
!sed -i 's,EXPERIMENT_DIR,'"$USER_EXPERIMENT_DIR"',' $SPECS_DIR/yolo_v4_train_resnet18_kitti.txt

# To enable QAT training on sample spec file, uncomment following lines
# !sed -i "s/enable_qat: false/enable_qat: true/g" $SPECS_DIR/yolo_train_resnet18_kitti.txt
# !sed -i "s/enable_qat: false/enable_qat: true/g" $SPECS_DIR/yolo_retrain_resnet18_kitti.txt

In [56]:
# By default, the sample spec file disables QAT training. You can force non-QAT training by running lines below
# !sed -i "s/enable_qat: true/enable_qat: false/g" $SPECS_DIR/yolo_train_resnet18_kitti.txt
# !sed -i "s/enable_qat: true/enable_qat: false/g" $SPECS_DIR/yolo_retrain_resnet18_kitti.txt

In [14]:
!cat $SPECS_DIR/yolo_v4_train_resnet34_kitti.txt

random_seed: 42
yolov4_config {
  big_anchor_shape: "[(316.00, 406.06),(406.47, 324.00),(431.00, 433.10)]"
  mid_anchor_shape: "[(170.91, 293.00),(327.96, 250.89),(239.27, 367.87)]"
  small_anchor_shape: "[(56.00, 66.00),(131.00, 147.00),(223.00, 203.00)]"
  box_matching_iou: 0.25
  arch: "resnet"
  nlayers: 34
  arch_conv_blocks: 2
  loss_loc_weight: 0.8
  loss_neg_obj_weights: 100.0
  loss_class_weights: 0.5
  label_smoothing: 0.0
  big_grid_xy_extend: 0.05
  mid_grid_xy_extend: 0.1
  small_grid_xy_extend: 0.2
  freeze_bn: false
  #freeze_blocks: 0
  force_relu: false
}
training_config {
  batch_size_per_gpu: 4
  num_epochs: 20
  enable_qat: false
  checkpoint_interval: 1
  learning_rate {
    soft_start_cosine_annealing_schedule {
      min_learning_rate: 1e-7
      max_learning_rate: 1e-4
      soft_start: 0.3
    }
  }
  regularizer {
    type: L1
    weight: 3e-5
  }
  optimizer {
    adam {
      epsilon: 1e-7
      beta1: 0.9
      beta2: 0.999
      amsgrad: false
    }
  }
  

## 3. Run TLT training <a class="anchor" id="head-3"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [3]:
!mkdir -p $DATA_DOWNLOAD_DIR/yolo_v4

In [8]:
%env LOGS=logs-yolov4-resnet34.txt

env: LOGS=logs-yolov4-resnet34.txt


In [ ]:
!cat $SPECS_DIR/yolo_v4_train_resnet34_kitti.txt >> $DATA_DOWNLOAD_DIR/kitti/final-test/$LOGS

In [9]:
!mkdir -p $DATA_DOWNLOAD_DIR/yolo_v4/weights

In [4]:
!ls /workspace/tlt-experiments/insect-thesis/data/faster_rcnn

darknet_19.hdf5    resnet_10.hdf5  resnet_34.hdf5
mobilenet_v2.hdf5  resnet_18.hdf5  resnet_50.hdf5


In [9]:
print("To run with multigpu, please change --gpus based on the number of available GPUs in your machine.")
!yolo_v4 train -e $SPECS_DIR/yolo_v4_train_resnet34_kitti.txt \
               -r $DATA_DOWNLOAD_DIR/yolo_v4 \
               -k $KEY \
               --use_amp \
               --gpus 1 \
               2> /dev/null \
               | stdbuf -o0 -i0 grep -P "mAP|Epoch|noctuidae|geometridae|coleoptera|diptera|odonata|orthoptera|hemiptera|hymenoptera|trichoptera|\d{0,1}\d00\/9797|Validation|Class" \
               | stdbuf -o0 -i0 tee -a $DATA_DOWNLOAD_DIR/$LOGS

To run with multigpu, please change --gpus based on the number of available GPUs in your machine.
Epoch 1/20
9700/9797 [============================>.] - ETA: 14s - loss: 40.7135
Epoch 00001: saving model to /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/yolo_v4/weights/yolov4_resnet34_epoch_001.tlt
coleoptera    AP    0.28965
diptera       AP    0.25492
geometridae   AP    0.56967
hemiptera     AP    0.38153
hymenoptera   AP    0.28093
noctuidae     AP    0.52177
odonata       AP    0.55519
orthoptera    AP    0.479
trichoptera   AP    0.6042
              mAP   0.43743
Validation loss: 5.086988793582504
Epoch 2/20
1000/9797 [==>...........................] - ETA: 21:00 - loss: 16.6109


9700/9797 [============================>.] - ETA: 13s - loss: 14.1535
Epoch 00002: saving model to /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/yolo_v4/weights/yolov4_resnet34_epoch_002.tlt
coleoptera    AP    0.61981
diptera       AP    0.62226
geometridae   AP    0.80607
hemiptera     AP    0.65858
hymenoptera   AP    0.59026
noctuidae     AP    0.81423
odonata       AP    0.6767
orthoptera    AP    0.72763
trichoptera   AP    0.69571
              mAP   0.69014
Validation loss: 2.5013627186930942
Epoch 3/20
2100/9797 [=====>........................] - ETA: 18:19 - loss: 11.8389


9700/9797 [============================>.] - ETA: 13s - loss: 10.8182
Epoch 00003: saving model to /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/yolo_v4/weights/yolov4_resnet34_epoch_003.tlt
coleoptera    AP    0.70428
diptera       AP    0.63326
geometridae   AP    0.65924
hemiptera     AP    0.71086
hymenoptera   AP    0.64561
noctuidae     AP    0.63252
odonata       AP    0.75309
orthoptera    AP    0.82116
trichoptera   AP    0.85342
              mAP   0.7126
Validation loss: 2.3414843148959577
Epoch 4/20
3300/9797 [=========>....................] - ETA: 15:27 - loss: 9.2415


9700/9797 [============================>.] - ETA: 13s - loss: 8.6855
Epoch 00004: saving model to /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/yolo_v4/weights/yolov4_resnet34_epoch_004.tlt
coleoptera    AP    0.58418
diptera       AP    0.70772
geometridae   AP    0.84757
hemiptera     AP    0.65083
hymenoptera   AP    0.63513
noctuidae     AP    0.85823
odonata       AP    0.7254
orthoptera    AP    0.76114
trichoptera   AP    0.75351
              mAP   0.72486
Validation loss: 2.360789888979483
Epoch 5/20
4600/9797 [=============>................] - ETA: 12:22 - loss: 7.5865


9700/9797 [============================>.] - ETA: 13s - loss: 7.3267
Epoch 00005: saving model to /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/yolo_v4/weights/yolov4_resnet34_epoch_005.tlt
coleoptera    AP    0.66249
diptera       AP    0.47357
geometridae   AP    0.81302
hemiptera     AP    0.65652
hymenoptera   AP    0.63347
noctuidae     AP    0.79236
odonata       AP    0.81172
orthoptera    AP    0.70003
trichoptera   AP    0.79905
              mAP   0.70469
Validation loss: 2.439706393527393
Epoch 6/20
5900/9797 [=================>............] - ETA: 9:16 - loss: 6.6373


...

9700/9797 [============================>.] - ETA: 13s - loss: 3.8407
Epoch 00015: saving model to /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/yolo_v4/weights/yolov4_resnet34_epoch_015.tlt
coleoptera    AP    0.80087
diptera       AP    0.80718
geometridae   AP    0.8919
hemiptera     AP    0.80741
hymenoptera   AP    0.7739
noctuidae     AP    0.88992
odonata       AP    0.88722
orthoptera    AP    0.85228
trichoptera   AP    0.88421
              mAP   0.84388
Validation loss: 1.8186113772298635
Epoch 16/20
7300/9797 [=====================>........] - ETA: 5:54 - loss: 3.7015


9700/9797 [============================>.] - ETA: 13s - loss: 3.6894
Epoch 00016: saving model to /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/yolo_v4/weights/yolov4_resnet34_epoch_016.tlt
coleoptera    AP    0.79229
diptera       AP    0.84688
geometridae   AP    0.89438
hemiptera     AP    0.80533
hymenoptera   AP    0.81723
noctuidae     AP    0.89033
odonata       AP    0.89088
orthoptera    AP    0.85536
trichoptera   AP    0.89523
              mAP   0.85421
Validation loss: 1.745060818501717
Epoch 17/20
8600/9797 [=========================>....] - ETA: 2:49 - loss: 3.5583


9700/9797 [============================>.] - ETA: 13s - loss: 3.5516
Epoch 00017: saving model to /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/yolo_v4/weights/yolov4_resnet34_epoch_017.tlt
coleoptera    AP    0.79861
diptera       AP    0.83618
geometridae   AP    0.89822
hemiptera     AP    0.81545
hymenoptera   AP    0.82246
noctuidae     AP    0.90279
odonata       AP    0.89081
orthoptera    AP    0.82884
trichoptera   AP    0.89019
              mAP   0.85373
Validation loss: 1.7361444032173066
Epoch 18/20
9700/9797 [============================>.] - ETA: 13s - loss: 3.4248
Epoch 00018: saving model to /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/yolo_v4/weights/yolov4_resnet34_epoch_018.tlt


coleoptera    AP    0.82892
diptera       AP    0.86604
geometridae   AP    0.90343
hemiptera     AP    0.83363
hymenoptera   AP    0.83146
noctuidae     AP    0.89714
odonata       AP    0.88424
orthoptera    AP    0.88265
trichoptera   AP    0.89287
              mAP   0.86893
Validation loss: 1.6764436389670823
Epoch 19/20
9700/9797 [============================>.] - ETA: 13s - loss: 3.2975
Epoch 00019: saving model to /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/yolo_v4/weights/yolov4_resnet34_epoch_019.tlt
coleoptera    AP    0.82949
diptera       AP    0.86658
geometridae   AP    0.90256
hemiptera     AP    0.82588
hymenoptera   AP    0.85229
noctuidae     AP    0.9019
odonata       AP    0.89631
orthoptera    AP    0.88052
trichoptera   AP    0.89953
              mAP   0.87278
Validation loss: 1.6135612700885733
Epoch 20/20
 800/9797 [=>............................] - ETA: 21:11 - loss: 3.2293


9700/9797 [============================>.] - ETA: 13s - loss: 3.1761
Epoch 00020: saving model to /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/yolo_v4/weights/yolov4_resnet34_epoch_020.tlt
coleoptera    AP    0.82905
diptera       AP    0.85969
geometridae   AP    0.90184
hemiptera     AP    0.84377
hymenoptera   AP    0.8552
noctuidae     AP    0.90046
odonata       AP    0.89624
orthoptera    AP    0.86595
trichoptera   AP    0.88239
              mAP   0.87051
Validation loss: 1.6676979792442517


In [ ]:
print("To resume from checkpoint, please change pretrain_model_path to resume_model_path in config file.")

In [65]:
print('Model for each epoch:')
print('---------------------')
!ls -ltrh $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights

Model for each epoch:
---------------------
total 774M
-rw-r--r-- 1 root root 232M Mar 30 09:35 yolov4_resnet18_epoch_010.tlt
-rw-r--r-- 1 root root 232M Mar 30 10:13 yolov4_resnet18_epoch_020.tlt
-rw-r--r-- 1 root root  78M Mar 30 11:09 yolov4_resnet18_epoch_025.tlt
-rw-r--r-- 1 root root  78M Mar 30 11:26 yolov4_resnet18_epoch_030.tlt
-rw-r--r-- 1 root root  78M Mar 30 11:45 yolov4_resnet18_epoch_035.tlt
-rw-r--r-- 1 root root  78M Mar 30 12:04 yolov4_resnet18_epoch_040.tlt


In [12]:
# Now check the evaluation stats in the csv file and pick the model with highest eval accuracy.
!cat $USER_EXPERIMENT_DIR/experiment_dir_unpruned/yolov4_training_log_resnet18.csv
%set_env EPOCH=010

epoch,AP_coleoptera,AP_diptera,AP_geometridae,AP_noctuidae,loss,lr,mAP,validation_loss
1,0.010987527527321593,0.045566402077420236,0.002746770402670313,0.039775349000553156,61.804099213814624,8.425e-06,0.024769012251991324,17.05926891034477
2,0.3850855776989521,0.445422088529778,0.3897902871937887,0.5192997974867524,19.06776086305317,1.6749998e-05,0.43489943772731776,4.485714787352609
3,0.5723589421640828,0.7134163188290134,0.568712942423448,0.8625999564197654,12.823797789943276,2.5074998e-05,0.6792720399590774,2.4730444934016975
4,0.7140295690283561,0.6305972139084953,0.6625141067083372,0.8440913058799258,11.332257657393313,3.3399996e-05,0.7128080488812787,2.060204979677225
5,0.6721608037633762,0.7688224050762745,0.7707369635880601,0.8699115940159656,10.387087464218505,4.1724998e-05,0.7704079416109191,2.1629225449968676
6,0.6851877249157674,0.7638337224721717,0.7235427236743687,0.8733834333103432,9.438171020124518,5.0049995e-05,0.7614869010931627,2.1832635448884594
7,0.758690972405281

## 4. Evaluate trained models <a class="anchor" id="head-4"></a>

In [13]:
!yolo_v4 evaluate -e $SPECS_DIR/yolo_v4_train_resnet18_kitti.txt \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/yolov4_resnet18_epoch_$EPOCH.tlt \
                  -k $KEY

Using TensorFlow backend.
Using TensorFlow backend.

2021-04-04 13:57:39,751 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.


2021-04-04 13:57:39,752 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.


2021-04-04 13:57:39,754 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.


2021-04-04 13:57:41,479 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


2021-04-04 13:57:41,531 [WA

## 5. Prune trained models <a class="anchor" id="head-5"></a>
* Specify pre-trained model
* Equalization criterion (`Only for resnets as they have element wise operations or MobileNets.`)
* Threshold for pruning.
* A key to save and load the model
* Output directory to store the model

Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold value depends on the dataset and the model. `0.5` in the block below is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.

In [71]:
!mkdir -p $USER_EXPERIMENT_DIR/experiment_dir_pruned

In [83]:
!yolo_v4 prune -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/yolov4_resnet18_epoch_$EPOCH.tlt \
               -e $SPECS_DIR/yolo_v4_train_resnet18_kitti.txt \
               -o $USER_EXPERIMENT_DIR/experiment_dir_pruned/yolov4_resnet18_pruned.tlt \
               -eq intersection \
               -pth 0.01 \
               -k $KEY

Using TensorFlow backend.
Using TensorFlow backend.

2021-03-30 13:19:04,222 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


2021-03-30 13:19:04,273 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.


2021-03-30 13:19:04,283 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.


2021-03-30 13:19:04,291 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.


2021-03-30 13:19:04,295 [WARNING] tensorflow: From /u

In [84]:
!ls -rlt $USER_EXPERIMENT_DIR/experiment_dir_pruned/

total 64748
-rw-r--r-- 1 root root 66298592 Mar 30 13:20 yolov4_resnet18_pruned.tlt


## 6. Retrain pruned models <a class="anchor" id="head-6"></a>
* Model needs to be re-trained to bring back accuracy after pruning
* Specify re-training specification
* WARNING: training will take several hours or one day to complete

In [85]:
# Printing the retrain spec file. 
# Here we have updated the spec file to include the newly pruned model as a pretrained weights.
!sed -i 's,EXPERIMENT_DIR,'"$USER_EXPERIMENT_DIR"',' $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt
!cat $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt

random_seed: 42
yolov4_config {
  big_anchor_shape: "[(662.73, 374.53),(879.24, 481.50),(703.77, 663.88)]"
  mid_anchor_shape: "[(272.97, 444.13), (392.70, 343.68), (442.03, 453.12)]"
  small_anchor_shape: "[(156.85, 199.63), (194.99, 367.89),(297.06, 245.06)]"
  box_matching_iou: 0.25
  arch: "resnet"
  nlayers: 18
  arch_conv_blocks: 2
  loss_loc_weight: 0.8
  loss_neg_obj_weights: 100.0
  loss_class_weights: 0.5
  label_smoothing: 0.0
  big_grid_xy_extend: 0.05
  mid_grid_xy_extend: 0.1
  small_grid_xy_extend: 0.2
  freeze_bn: false
  #freeze_blocks: 0
  force_relu: false
}
training_config {
  batch_size_per_gpu: 2
  num_epochs: 40
  enable_qat: false
  checkpoint_interval: 2
  learning_rate {
    soft_start_cosine_annealing_schedule {
      min_learning_rate: 1e-7
      max_learning_rate: 1e-4
      soft_start: 0.3
    }
  }
  regularizer {
    type: NO_REG
    weight: 3e-9
  }
  optimizer {
    adam {
      epsilon: 1e-7
      beta1: 0.9
      beta2: 0.999
      amsgrad: false
   

In [86]:
!mkdir -p $USER_EXPERIMENT_DIR/experiment_dir_retrain

In [87]:
# Retraining using the pruned model as pretrained weights 
!yolo_v4 train --gpus 1 \
               --use_amp \
               -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt \
               -r $USER_EXPERIMENT_DIR/experiment_dir_retrain \
               -k $KEY

Using TensorFlow backend.
Using TensorFlow backend.

2021-03-30 13:20:32,360 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/horovod/tensorflow/__init__.py:117: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.


2021-03-30 13:20:32,360 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/horovod/tensorflow/__init__.py:143: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.


2021-03-30 13:20:32,479 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/yolo_v4/scripts/train.py:49: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.


2021-03-30 13:20:32,479 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8-


2021-03-30 13:20:42,534 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.


2021-03-30 13:20:43,433 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

Epoch 1/40
   2/3044 [..............................] - ETA: 5:44:19 - loss: 4.0661/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (1.264549). Check your callbacks.
  % delta_t_median)
3044/3044 [==============================] - 281s 92ms/step - loss: 2.9767
Epoch 2/40
3044/3044 [==============================] - 262s 86ms/step - loss: 2.4707

Epoch 00002: saving model to /workspace/tlt-experiments/insect-thesis/experiment_dir_retrain/weights/yolov4_resnet18_epoch_002.tlt
Producing pre

In [79]:
# Listing the newly retrained model.
!ls -rlt $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights

total 475080
-rw-r--r-- 1 root root 162160480 Mar 30 12:35 yolov4_resnet18_epoch_005.tlt
-rw-r--r-- 1 root root 162160480 Mar 30 12:56 yolov4_resnet18_epoch_010.tlt
-rw-r--r-- 1 root root 162160480 Mar 30 13:17 yolov4_resnet18_epoch_015.tlt


In [80]:
# Now check the evaluation stats in the csv file and pick the model with highest eval accuracy.
!cat $USER_EXPERIMENT_DIR/experiment_dir_retrain/yolov4_training_log_resnet18.csv
%set_env EPOCH=040

epoch,AP_geometridae,AP_noctuidae,loss,lr,mAP,validation_loss
1,nan,nan,2.9699207752864716,8.425e-06,nan,nan
2,nan,nan,2.4719353376174253,1.6749998e-05,nan,nan
3,nan,nan,2.426941933303719,2.5074998e-05,nan,nan
4,nan,nan,2.4175226155193656,3.3399996e-05,nan,nan
5,0.40039567749065,0.9050360334235674,2.4868202339294863,4.1724998e-05,0.6527158554571088,2.13381304120171
6,nan,nan,2.506702417906731,5.0049995e-05,nan,nan
7,nan,nan,2.5245346796261967,5.8374997e-05,nan,nan
8,nan,nan,2.5163331839908283,6.67e-05,nan,nan
9,nan,nan,2.5604243331176377,7.5025e-05,nan,nan
10,0.4346957101610327,0.8968245366205689,2.5762420302739124,8.3349994e-05,0.6657601233908008,2.211998022519625
11,nan,nan,2.5954854175468,9.167499e-05,nan,nan
12,nan,nan,2.624354499829113,9.999999e-05,nan,nan
13,nan,nan,2.5883319928517636,9.984602e-05,nan,nan
14,nan,nan,2.5596985588444987,9.938503e-05,nan,nan
15,0.4190668131475363,0.8742922661585135,2.5287796151230744,9.861987e-05,0.6466795396530249,2.247077383233245
env: EPOCH=040


## 7. Evaluate retrained model <a class="anchor" id="head-7"></a>

In [81]:
!yolo_v4 evaluate -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/yolov4_resnet18_epoch_$EPOCH.tlt \
                  -k $KEY

Using TensorFlow backend.
Using TensorFlow backend.

2021-03-30 13:18:20,302 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.


2021-03-30 13:18:20,303 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.


2021-03-30 13:18:20,305 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

Traceback (most recent call last):
  File "/home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/yolo_v4/scripts/evaluate.p

## 8. Visualize inferences <a class="anchor" id="head-8"></a>
In this section, we run the tlt-infer tool to generate inferences on the trained models and visualize the results.

In [20]:
# Copy some test images
#!mkdir -p /workspace/tlt-experiments/insect-thesis/yolo_v4/test_samples
#!cp $DATA_DOWNLOAD_DIR/kitti/fusion2/inference/images/* /workspace/tlt-experiments/insect-thesis/yolo_v4/test_samples/

In [14]:
# remove previous inference results
!rm /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/yolo_v4/inference_*/*

In [15]:
# Running inference for detection on n images
!yolo_v4 inference -i /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/val/images \
                   -o /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/yolo_v4/inference_images \
                   -e $SPECS_DIR/yolo_v4_train_resnet18_kitti.txt \
                   -m /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/yolo_v4/weights/yolov4_resnet18_epoch_020.tlt \
                   -l /workspace/tlt-experiments/insect-thesis/data/kitti/final-test/yolo_v4/inference_labels \
                   -k $KEY

Using TensorFlow backend.
Using TensorFlow backend.

2021-05-15 17:35:22,475 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.


2021-05-15 17:35:22,475 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.


2021-05-15 17:35:22,477 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.


2021-05-15 17:35:24,953 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


2021-05-15 17:35:24,973 [WA

The `tlt-infer` tool produces two outputs. 
1. Overlain images in `$USER_EXPERIMENT_DIR/yolo_infer_images`
2. Frame by frame bbox labels in kitti format located in `$USER_EXPERIMENT_DIR/yolo_infer_labels`

In [ ]:
# Simple grid visualizer
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['USER_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'yolo_infer_images' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

In [17]:
## 8bis. KITTI TO YOLO for metrics <a class="anchor" id="head-9"></a>

In [16]:
!rm ../data/kitti/final-test/val/labels_yolo/*
!python3 ../scripts/KITTI_to_YOLO.py validation ../data/kitti/final-test/val ../data/kitti/final-test/val/labels_yolo

100%|██████████████████████████████████████| 4354/4354 [00:13<00:00, 314.98it/s]


In [17]:
!rm ../data/kitti/final-test/yolo_v4/inference_labels_yolo/*
!python3 ../scripts/KITTI_to_YOLO.py inference ../data/kitti/final-test/yolo_v4 ../data/kitti/final-test/yolo_v4/inference_labels_yolo

rm: cannot remove '../data/kitti/final-test/yolo_v4/inference_labels_yolo/*': No such file or directory
100%|██████████████████████████████████████| 4354/4354 [00:09<00:00, 447.84it/s]


In [18]:
!chmod -R a+rw /workspace/tlt-experiments/insect-thesis

In [21]:
!rm ../data/kitti/final-test/yolo_v4/stats-18-cmd/*

In [22]:
!python3 /workspace/tlt-experiments/Object-Detection-Metrics/pascalvoc.py \
        -gt ../insect-thesis/data/kitti/final-test/val/labels_yolo/ \
        -det ../insect-thesis/data/kitti/final-test/yolo_v4/inference_labels_yolo/ \
        -sp ../insect-thesis/data/kitti/final-test/yolo_v4/stats-18-cmd
        

Figure(640x480)
Figure(640x480)
Figure(640x480)
Figure(640x480)
Figure(640x480)
Figure(640x480)
Figure(640x480)
Figure(640x480)
Figure(640x480)
AP: 93.93% (0)
AP: 90.17% (1)
AP: 86.40% (2)
AP: 81.50% (3)
AP: 92.39% (4)
AP: 89.97% (5)
AP: 85.39% (6)
AP: 84.00% (7)
AP: 92.40% (8)
mAP: 88.46%


## 9. Deploy! <a class="anchor" id="head-9"></a>

If you trained a non-QAT model, you may export in FP32, FP16 or INT8 mode using the code block below. For INT8, you need to provide calibration image directory.

In [ ]:
# tlt-export will fail if .etlt already exists. So we clear the export folder before tlt-export
!rm -rf $USER_EXPERIMENT_DIR/export
!mkdir -p $USER_EXPERIMENT_DIR/export
# Export in FP32 mode. Change --data_type to fp16 for FP16 mode
!yolo_v4 export -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/yolov4_resnet18_epoch_$EPOCH.tlt \
                -k $KEY \
                -o $USER_EXPERIMENT_DIR/export/yolov4_resnet18_epoch_$EPOCH.etlt \
                -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt \
                --batch_size 16 \
                --data_type fp32

# Uncomment to export in INT8 mode (generate calibration cache file). 
# !yolo_v4 export -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/yolov4_resnet18_epoch_$EPOCH.tlt  \
#                 -o $USER_EXPERIMENT_DIR/export/yolov4_resnet18_epoch_$EPOCH.etlt \
#                 -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt \
#                 -k $KEY \
#                 --cal_image_dir  $USER_EXPERIMENT_DIR/data/testing/image_2 \
#                 --data_type int8 \
#                 --batch_size 16 \
#                 --batches 10 \
#                 --cal_cache_file $USER_EXPERIMENT_DIR/export/cal.bin  \
#                 --cal_data_file $USER_EXPERIMENT_DIR/export/cal.tensorfile

`Note:` In this example, for ease of execution we restrict the number of calibrating batches to 10. TLT recommends the use of at least 10% of the training dataset for int8 calibration.

If you train a QAT model, you may only export in INT8 mode using following code block. This generates an etlt file and the corresponding calibration cache. You can throw away the calibration cache and just use the etlt file in tlt-converter or DeepStream for FP32 or FP16 mode. But please note this gives sub-optimal results. If you want to deploy in FP32 or FP16, you should disable QAT in training.

In [ ]:
# Uncomment to export QAT model in INT8 mode (generate calibration cache file).
# !rm -rf $USER_EXPERIMENT_DIR/export
# !mkdir -p $USER_EXPERIMENT_DIR/export
# !yolo_v4 export -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/yolov4_resnet18_epoch_$EPOCH.tlt  \
#                 -o $USER_EXPERIMENT_DIR/export/yolov4_resnet18_epoch_$EPOCH.etlt \
#                 -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt \
#                 -k $KEY \
#                 --data_type int8 \
#                 --cal_cache_file $USER_EXPERIMENT_DIR/export/cal.bin

In [ ]:
print('Exported model:')
print('------------')
!ls -lh $USER_EXPERIMENT_DIR/export

Verify engine generation using the `tlt-converter` utility included with the docker.

The `tlt-converter` produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please instantiate this docker and execute the `tlt-converter` command, with the exported `.etlt` file and calibration cache (for int8 mode) on your target device. The converter utility included in this docker only works for x86 devices, with discrete NVIDIA GPU's. 

For the jetson devices, please download the converter for jetson from the dev zone link [here](https://developer.nvidia.com/tlt-converter). 

If you choose to integrate your model into deepstream directly, you may do so by simply copying the exported `.etlt` file along with the calibration cache to the target device and updating the spec file that configures the `gst-nvinfer` element to point to this newly exported model. Usually this file is called `config_infer_primary.txt` for detection models and `config_infer_secondary_*.txt` for classification models.

In [ ]:
# Convert to TensorRT engine (FP32)
!tlt-converter -k $KEY \
               -d 3,384,1248 \
               -o BatchedNMS \
               -e $USER_EXPERIMENT_DIR/export/trt.engine \
               -m 16 \
               -t fp32 \
               -i nchw \
               $USER_EXPERIMENT_DIR/export/yolov4_resnet18_epoch_$EPOCH.etlt

# Convert to TensorRT engine (FP16)
# !tlt-converter -k $KEY \
#                -d 3,384,1248 \
#                -o BatchedNMS \
#                -e $USER_EXPERIMENT_DIR/export/trt.engine \
#                -m 16 \
#                -t fp16 \
#                -i nchw \
#                $USER_EXPERIMENT_DIR/export/yolov4_resnet18_epoch_$EPOCH.etlt

# Convert to TensorRT engine (INT8).
# !tlt-converter -k $KEY  \
#                -d 3,384,1248 \
#                -o BatchedNMS \
#                -c $USER_EXPERIMENT_DIR/export/cal.bin \
#                -e $USER_EXPERIMENT_DIR/export/trt.engine \
#                -b 8 \
#                -m 16 \
#                -t int8 \
#                -i nchw \
#                $USER_EXPERIMENT_DIR/export/yolov4_resnet18_epoch_$EPOCH.etlt

In [ ]:
print('Exported engine:')
print('------------')
!ls -lh $USER_EXPERIMENT_DIR/export/trt.engine

## 10. Verify the deployed model <a class="anchor" id="head-10"></a>
Verify the converted engine by visualizing TensorRT inferences.


In [ ]:
# Infer using TensorRT engine

!yolo_v4 inference -m $USER_EXPERIMENT_DIR/export/trt.engine \
                   -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt \
                   -i /workspace/examples/yolo_v4/test_samples \
                   -o $USER_EXPERIMENT_DIR/yolo_infer_images \
                   -t 0.6

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'yolo_infer_images' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)